See example: https://github.com/ESGF/esgf-compute-api

In [45]:
from owslib.wps import WebProcessingService

In [46]:
api_key = 'TOKEN'

In [47]:
#client = WebProcessingService('https://aims2.llnl.gov/wps/', api_key=api_key, verify=False)
client = WebProcessingService('https://bovec.dkrz.de/ows/proxy/emu', api_key=api_key, verify=False)

/Users/pingu/.conda/envs/owslib/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [48]:
for p in client.processes:
    print(p.identifier)

ultimate_question
sleep
nap
bbox
hello
dummyprocess
wordcounter
chomsky
inout
binaryoperatorfornumbers
show_error
multiple_outputs
esgf_demo
output_formats
poly_centroid
ncmeta
CDAT.subset


In [49]:
proc = client.describeprocess('CDAT.subset')
proc.identifier


'CDAT.subset'

In [50]:
from owslib.wps import Variable

In [51]:
files = [
    # (122, 240, 480)
    'http://aims3.llnl.gov/thredds/dodsC/cmip5_css02_data/cmip5/output1/CMCC/CMCC-CM/decadal2005/mon/atmos/Amon/r1i2p1/tas/1/tas_Amon_CMCC-CM_decadal2005_r1i2p1_200511-201512.nc',
    # (120, 240, 480)
    'http://aims3.llnl.gov/thredds/dodsC/cmip5_css02_data/cmip5/output1/CMCC/CMCC-CM/decadal2005/mon/atmos/Amon/r1i2p1/tas/1/tas_Amon_CMCC-CM_decadal2005_r1i2p1_201601-202512.nc',
    # (120, 240, 480)
    'http://aims3.llnl.gov/thredds/dodsC/cmip5_css02_data/cmip5/output1/CMCC/CMCC-CM/decadal2005/mon/atmos/Amon/r1i2p1/tas/1/tas_Amon_CMCC-CM_decadal2005_r1i2p1_202601-203512.nc'
]


In [52]:
inputs = [('variable', Variable(uri=x, id='tas')) for x in files]

In [53]:
exec = client.execute('CDAT.subset', inputs=inputs)

In [54]:
exec.isComplete()

True

In [55]:
output = exec.processOutputs[0]
print(output.reference)

https://bovec.dkrz.de/download/outputs/emu/ccf587c0-1b49-11e9-965a-109836a7cf3a/input.txt
